In [ ]:
from clickhouse_driver import Client

client = Client('oleg.orbita.work', port = 9000, user='student_ml',
password='CGRV8zemLGgsfdz7Uc6ZJeisGTcPQa', settings={'use_numpy': True})
client.execute('SHOW DATABASES')

In [ ]:
table_name = 'students_datas.game_sales'
# .drop(labels=['Name', 'Genre', 'Year', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales'], axis=1)
%config InlineBackend.figure_format = 'svg' 

In [ ]:
df = client.query_dataframe(f"SELECT * FROM {table_name}")
columns = df.columns
df.head(30)

In [ ]:
df = df.dropna()
df.head(30)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np 
import seaborn as sns

In [ ]:
# Игры каких жанров были наиболее популярны до 2000 года, а какие после?
# Оцените популярность жанров по количеству выпущенных игр и по объему продаж по всему миру.
df.loc[df['Year'] < 2000]['Genre'].value_counts().sort_values(ascending=False).plot(kind='bar')

In [ ]:
df.loc[df['Year'] < 2000].groupby('Genre')['Global_Sales'].sum().sort_values(ascending=False).plot(kind='bar', color='crimson')

In [ ]:
df.loc[df['Year'] >= 2000]['Genre'].value_counts().sort_values(ascending=False).plot(kind='bar')

In [ ]:
df.loc[df['Year'] >= 2000].groupby('Genre')['Global_Sales'].sum().sort_values(ascending=False).plot(kind='bar', color='crimson')

#### Самые популярные жанры по выпуску игр: Спортивные и Экшен 

#### Самые популярные жанры по продажам: (Платформеры и ролевые) (Спортивные и Экшн)

In [ ]:
df.loc[df['Year'] < 2000].groupby('Genre')['Global_Sales'].mean().sort_values(ascending=False).plot(kind='bar', color='gold')

#### Платформеры выпускаются меньше, но, судя по всему, у жанра есть своя аудитория, готовая тратить большие деньги, и единицы компаний разработки, которые ставят высокие цены, производя небольшое количество игр.
#### Предположу, что это Нинтендо

In [ ]:
df.loc[df['Year'] < 2000].groupby('Publisher')['Global_Sales'].sum().sort_values(ascending=False).head(10).plot(kind='bar', color='tomato')

###### ну кто-ж ещё...

In [ ]:
df.loc[df['Year'] >= 2000].groupby('Publisher')['Global_Sales'].sum().sort_values(ascending=False).head(10).plot(kind='bar', color='tomato')

##### положение в общем и целом не изменилось, добавились другие гиганты индустрии, которые также сделали игры, положившие начало новым жанрам

In [ ]:
df.loc[df['Year'] >= 2000].groupby('Name')['Global_Sales'].sum().sort_values(ascending=False).head(30).plot(kind='bar', color='orange')

###### марио, калл оф дути, марио, калл оф дути, марио, калл оф дути... 

In [ ]:
# Отобразите на графике общее число видеоигр, выпущенных в каждом году.
df.drop_duplicates(subset=['Name'])['Year'].dropna().astype(int).value_counts().sort_index().plot(kind='bar')

In [ ]:
# Определите трех издателей, выпустивших наибольшее количество видеоигр.
# Изобразите количество выпущенных издателями видеоигр для каждой платформы
# на столбиковой диаграмме (можно использовать диаграмму с накоплением).
import pandas as pd

table = df[df['Publisher'].isin(df['Publisher'].value_counts().head(3).index.tolist())]
pd.crosstab(table['Publisher'], table['Platform']).plot(kind='bar', stacked=True)
plt.legend(title='Platforms', bbox_to_anchor=(1, 1))
plt.show()

In [ ]:
# Отобразите на круговых диаграммах доли суммарного объема продаж с 1980г. до 2000г. и с 2000г. до 2020г.
# в Северной Америке, Европе, Японии от объема продаж по всему миру.
group_1980_2000 = df[(df['Year'] >= 1980) & (df['Year'] < 2000)]
plt.pie(
    x=[np.sum(group_1980_2000.NA_Sales), np.sum(group_1980_2000.EU_Sales),
       np.sum(group_1980_2000.JP_Sales), np.sum(group_1980_2000.Other_Sales)],
    labels=["NA", "EU", "JP", "Other"],
    autopct='%1.2f%%')
plt.title('Продажи 1980-2000')
plt.show()

plt.pie(
    x=[np.sum(group_1980_2000.JP_Sales), np.sum(group_1980_2000.Global_Sales)],
    labels=["JP", "Global"],
    autopct='%1.2f%%',
    colors=['Aqua', 'Lime'])
plt.title('Продажи Япония - Мир\n1980-2000')
plt.show()

plt.pie(
    x=[np.sum(group_1980_2000.EU_Sales), np.sum(group_1980_2000.Global_Sales)],
    labels=["EU", "Global"],
    autopct='%1.2f%%')
plt.title('Продажи 1980-2000\nЕвропа - Мир')
plt.show()

plt.pie(
    x=[np.sum(group_1980_2000.NA_Sales), np.sum(group_1980_2000.Global_Sales)],
    labels=["NA", "Global"],
    autopct='%1.2f%%',
    colors=['Tan', 'Silver'])
plt.title('Продажи 1980-2000\nСеверная Америка - Мир')
plt.show()

In [ ]:
group_2000_2020 =  df[(df['Year'] >= 2000) & (df['Year'] < 2020)]
plt.pie(
    x=[np.sum(group_2000_2020.NA_Sales), np.sum(group_2000_2020.EU_Sales),
       np.sum(group_2000_2020.JP_Sales), np.sum(group_2000_2020.Other_Sales)],
    labels=["NA", "EU", "JP", "Other"],
    autopct='%1.2f%%',
    colors=sns.color_palette('Set2'))
plt.title('Продажи 2000-2020')
plt.show()

plt.pie(
    x=[np.sum(group_2000_2020.JP_Sales), np.sum(group_2000_2020.Global_Sales)],
    labels=["JP", "Global"],
    autopct='%1.2f%%',
    colors=['Aqua', 'Lime'])
plt.title('Продажи Япония - Мир\n1980-2000')
plt.show()

plt.pie(
    x=[np.sum(group_2000_2020.EU_Sales), np.sum(group_2000_2020.Global_Sales)],
    labels=["EU", "Global"],
    autopct='%1.2f%%')
plt.title('Продажи 1980-2000\nЕвропа - Мир')
plt.show()

plt.pie(
    x=[np.sum(group_2000_2020.NA_Sales), np.sum(group_2000_2020.Global_Sales)],
    labels=["NA", "Global"],
    autopct='%1.2f%%',
    colors=['Tan', 'Silver'])
plt.title('Продажи 1980-2000\nСеверная Америка - Мир')
plt.show()